<a href="https://colab.research.google.com/github/taliahtarik/dsprojectjterm/blob/main/day2pt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import seaborn as sns
from google.colab import files
uploaded = files.upload()

Saving FY_2024_Hospital_Readmissions_Reduction_Program_Hospital.csv to FY_2024_Hospital_Readmissions_Reduction_Program_Hospital (1).csv


In [ ]:
file = "FY_2024_Hospital_Readmissions_Reduction_Program_Hospital.csv"
data = pd.read_csv(file)

# ERR Visualization by State

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

state_err = data.groupby(['State', 'Measure Name'])['Excess Readmission Ratio'].mean().reset_index()

def err_category(err):
    if err < 1:
        return 'Below Expected (Green)'
    elif err == 1:
        return 'At Expected (Yellow)'
    else:
        return 'Above Expected (Red)'

state_err['ERR Category'] = state_err['Excess Readmission Ratio'].apply(err_category)
state_err['Color'] = state_err['ERR Category'].map({
    'Below Expected (Green)': 'green',
    'At Expected (Yellow)': 'yellow',
    'Above Expected (Red)': 'red'
})

condition_name_mapping = {
    'READM-30-AMI-HRRP': 'Heart Attack (AMI)',
    'READM-30-HF-HRRP': 'Heart Failure (HF)',
    'READM-30-PN-HRRP': 'Pneumonia',
    'READM-30-COPD-HRRP': 'Chronic Obstructive Pulmonary Disease (COPD)',
    'READM-30-HIP-KNEE-HRRP': 'Hip/Knee Replacement (THA/TKA)',
    'READM-30-CABG-HRRP': 'Coronary Artery Bypass Graft (CABG) Surgery'
}


state_err['Measure Name'] = state_err['Measure Name'].replace(condition_name_mapping)


conditions = state_err['Measure Name'].unique()
figures = []


for condition in conditions:
    condition_data = state_err[state_err['Measure Name'] == condition]

    fig = go.Choropleth(
        locations=condition_data['State'],
        locationmode='USA-states',
        z=condition_data['Excess Readmission Ratio'],
        text=condition_data['State'],
        hoverinfo='text+z',
        colorscale='RdYlGn',
        zmin=0.85,  # Adjust based on your data
        zmax=1.05,  # Adjust based on your data
        showscale=True,
        colorbar=dict(
            title="Excess Readmission Ratio",
            titleside="right",
            tickvals=[0.96, 1, 1.04],
            ticktext=["Below Expected", "At Expected", "Above Expected"],
            ticks="outside"
        )
    )
    figures.append(fig)


choropleth_map = go.Figure()

for idx, fig in enumerate(figures):
    choropleth_map.add_trace(fig)

buttons = []
for idx, condition in enumerate(conditions):
    buttons.append(dict(
        method='update',
        label=condition,
        args=[{'visible': [i == idx for i in range(len(figures))]}]
    ))

choropleth_map.update_layout(
    title_text="Average Excess Readmission Ratio (ERR) by State and Condition",
    geo=dict(
        scope='usa',
        showcoastlines=True,
        coastlinecolor="Black",
        showland=True,
        landcolor="lightgray",
        projection_type="albers usa"
    ),
    updatemenus=[dict(
        active=0,
        buttons=buttons,
        x=0.5,
        xanchor='center',
        y=1.1,
        yanchor='top'
    )]
)

choropleth_map.show()
